Generamos los datos para una red de espines

In [ ]:
import numpy as np
import scipy.sparse as sp
import pandas as pd
np.random.seed(12)

### Tamaño de la cadena
L=40

# Creamos configuraciones aleatorias
states=np.random.choice([-1, 1], size=(10000,L))

def ising_energies(states):
    """
    Esta función calcula las energias de interacción.
    """
    L = states.shape[1]
    J = np.zeros((L, L),)
    #matriz que define la interacciçon entre vecinos cercanos con condiciones periódicas.
    #se podría  hacer con un doble ciclo pero es mucho menos eficiente!
    for i in range(L): 
        J[i,(i+1)%L]=-1.0 
        
    # calculamos las energias sumando las interacciones a primeros vecinos
    E = np.einsum('...i,ij,...j->...',states,J,states)

    return E,J
energies=ising_energies(states)[0]
J=ising_energies(states)[1]

In [ ]:
#reformamos los datos para que tengan la forma vectorial descrita
states=np.einsum('...i,...j->...ij',states,states)
#para cada configuración tenemos una matriz 40x40
shape=states.shape
#shape=(10000,40,40)
states=states.reshape((shape[0],shape[1]*shape[2]))
#states.shape =(10000,1600)
Data=[states,energies]

## Regresión lineal
Aplicamos la regresión lineal a los datos recien construidos

In [ ]:
#definimos el número de muestras
n_samples=500
#definimos la data de entrenamiento y  de test
X_train=Data[0][:n_samples] #shape(500,1600)
Y_train=Data[1][:n_samples]
X_test=Data[0][n_samples:3*n_samples//2]
Y_test=Data[1][n_samples:3*n_samples//2]

In [ ]:
from sklearn import linear_model
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn
%matplotlib inline

#definimos el modelo de regrasion lineal
leastsq=linear_model.LinearRegression()

# guardamos los errores para comparar
train_errors_leastsq = []
test_errors_leastsq = []

coefs_leastsq=[]

leastsq.fit(X_train,Y_train)
coefs_leastsq.append(leastsq.coef_)
train_errors_leastsq.append(leastsq.score(X_train,Y_train))
test_errors_leastsq.append(leastsq.score(X_test,Y_test))
prediccion=leastsq.predict(X_test)

#Dibujamos los resultados de la regresión lineal: J, pendiente de la regresion
J_leastsq=np.array(leastsq.coef_).reshape((L,L))

cmap_args=dict(vmin=-1., vmax=1., cmap="seismic")

fig, ax=plt.subplots()
im=ax.imshow(J_leastsq, **cmap_args)
ax.set_title('Entrenamiento$=%.3f$, Test$=%.3f$'%(train_errors_leastsq[-1], test_errors_leastsq[-1]),fontsize=16)

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05, add_to_figure=True)
cbar=fig.colorbar(im, cax=cax)
    
cbar.ax.set_yticklabels(np.arange(-1.0, 1.0+0.25, 0.25),fontsize=14)
cbar.set_label('$J_{i,j}$',labelpad=15, y=0.5,fontsize=20,rotation=0)

plt.savefig("regresion1.png")

Comporamos los resultados con el uso de regularizadores

In [ ]:
ridge=linear_model.Ridge()
lasso = linear_model.Lasso()

# define error lists
train_errors_ridge = []
test_errors_ridge = []

train_errors_lasso = []
test_errors_lasso = []

# set regularisation strength values
lmbdas = np.logspace(-4, 5, 10)

#Initialize coeffficients for ridge regression and Lasso
coefs_ridge = []
coefs_lasso=[]

for lmbda in lmbdas:
    
    ### apply RIDGE regression
    ridge.set_params(alpha=lmbda) # set regularisation parameter
    ridge.fit(X_train, Y_train) # fit model 
    coefs_ridge.append(ridge.coef_) # store weights
    # use the coefficient of determination R^2 as the performance of prediction.
    train_errors_ridge.append(ridge.score(X_train, Y_train))
    test_errors_ridge.append(ridge.score(X_test,Y_test))
    
    ### apply LASSO regression
    lasso.set_params(alpha=lmbda) # set regularisation parameter
    lasso.fit(X_train, Y_train) # fit model
    coefs_lasso.append(lasso.coef_) # store weights
    # use the coefficient of determination R^2 as the performance of prediction.
    train_errors_lasso.append(lasso.score(X_train, Y_train))
    test_errors_lasso.append(lasso.score(X_test,Y_test))

    ### plot Ising interaction J
    J_ridge=np.array(ridge.coef_).reshape((L,L))
    J_lasso=np.array(lasso.coef_).reshape((L,L))

    cmap_args=dict(vmin=-1., vmax=1., cmap='seismic')

    fig, axarr = plt.subplots(nrows=1, ncols=2)
    
    axarr[0].imshow(J_ridge,**cmap_args)
    axarr[0].set_title('Ridge $\lambda=%.4f$\n Train$=%.3f$, Test$=%.3f$' %(lmbda,train_errors_ridge[-1],test_errors_ridge[-1]),fontsize=16)
    axarr[0].tick_params(labelsize=16)
    
    im=axarr[1].imshow(J_lasso,**cmap_args)
    axarr[1].set_title('LASSO $\lambda=%.4f$\n Train$=%.3f$, Test$=%.3f$' %(lmbda,train_errors_lasso[-1],test_errors_lasso[-1]),fontsize=16)
    axarr[1].tick_params(labelsize=16)
    
    divider = make_axes_locatable(axarr[1])
    cax = divider.append_axes("right", size="5%", pad=0.05, add_to_figure=True)
    cbar=fig.colorbar(im, cax=cax)
    
    cbar.ax.set_yticklabels(np.arange(-1.0, 1.0+0.25, 0.25),fontsize=14)
    cbar.set_label('$J_{i,j}$',labelpad=15, y=0.5,fontsize=20,rotation=0)
    
    fig.subplots_adjust(right=2.0)
    plt.show()



In [ ]:
#dibujamos la gráfica con las precisiones de ambos modelos para comparar
plt.semilogx(lmbdas, train_errors_ridge,'r',label='Train (RG)',linewidth=1)
plt.semilogx(lmbdas, test_errors_ridge,'--r',label='Test (RG)',linewidth=1)
plt.semilogx(lmbdas, train_errors_lasso, 'b',label='Train (LASSO)')
plt.semilogx(lmbdas, test_errors_lasso, '--b',label='Test (LASSO)')

fig = plt.gcf()
fig.set_size_inches(10.0, 6.0)

#plt.vlines(alpha_optim, plt.ylim()[0], np.max(test_errors), color='k',
#           linewidth=3, label='Optimum on test')
plt.legend(loc='lower left',fontsize=16)
plt.ylim([-0.1, 1.1])
plt.xlim([min(lmbdas), max(lmbdas)])
plt.xlabel(r'$\lambda$',fontsize=16)
plt.ylabel('R²',fontsize=16)
plt.tick_params(labelsize=16)
plt.savefig("performance.png")
